# Query Data using LLM

Here is the overall RAG pipeline.   In this notebook, we will do steps (6), (7), (8), (9) and (10)
- Importing data is already done in this notebook [rag_2_load_data_into_milvus.ipynb](rag_2_load_data_into_milvus.ipynb)
- 👉 Step 6: Calculate embedding for user query
- 👉 Step 7 & 8: Send the query to vector db to retrieve relevant documents
- 👉 Step 9 & 10: Send the query and relevant documents (returned above step) to LLM and get answers to our query

![image missing](media/rag-overview-2.png)

## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

## Step-2: Connect to Vector Database

Milvus can be embedded and easy to use.

<span style="color:blue;">Note: If you encounter an error about unable to load database, try this: </span>

- <span style="color:blue;">In **vscode** : **restart the kernel** of previous notebook. This will release the db.lock </span>
- <span style="color:blue;">In **Jupyter**: Do `File --> Close and Shutdown Notebook` of previous notebook. This will release the db.lock</span>
- <span style="color:blue;">Re-run this cell again</span>


In [2]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=False  # so we load the index from db
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

/home/sujee/my-stuff/ai-alliance/data-prep-kit-examples/dpk-dev/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/sujee/my-stuff/ai-alliance/data-prep-kit-examples/dpk-dev/.venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptoo

✅ Connected to Milvus instance:  ./rag_1_dpk.db


## Step-3: Setup Embeddings

Use the same embeddings we used to index our documents!

In [3]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [4]:
from llama_index.embeddings.litellm import LiteLLMEmbedding
from llama_index.core import Settings

Settings.embed_model = LiteLLMEmbedding(
        model_name=MY_CONFIG.EMBEDDING_MODEL,
        embed_batch_size=50,  # Batch size for embedding (default is 10)
    )
print (f"✅ Using embedding model: {MY_CONFIG.EMBEDDING_MODEL}")


✅ Using embedding model: nebius/Qwen/Qwen3-Embedding-8B


In [5]:
## local embedding model
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings

# print ("✅ Using embedding Model:", MY_CONFIG.EMBEDDING_MODEL)
# print ("✅ Using embedding length:", MY_CONFIG.EMBEDDING_LENGTH)

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name = MY_CONFIG.EMBEDDING_MODEL
# )

## Step-4: Load Document Index from DB

In [6]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context)

print ("✅ Loaded index from vector db:", MY_CONFIG.DB_URI )

✅ Loaded index from vector db: ./rag_1_dpk.db
CPU times: user 757 μs, sys: 0 ns, total: 757 μs
Wall time: 734 μs


## Step-6: Using LLM

We can use LLMs running on remote services or locally (e.g. using Ollama).  We use [LiteLLM library](https://docs.litellm.ai/docs/) to choose LLM runtime.

Here are some examples.

- [Nebuis Token Factory](https://tokenfactory.nebius.com/)
- [replicate.com](https://replicate.com)

**How to use the LLM inference services**

If Using Nebius

Update `.env` file as follows

```ini
LLM_MODEL = 'nebius/openai/gpt-oss-120b'
NEBIUS_API_KEY = 'your key goes here'
```

If using Replicate

```ini
LLM_MODEL = 'ibm-granite/granite-3.3-8b-instruct'
REPLICATE_API_TOKEN=xyz
```


In [7]:
from llama_index.llms.litellm import LiteLLM

# Setup LLM
print (f"✅ Using LLM model : {MY_CONFIG.LLM_MODEL}")
Settings.llm = LiteLLM (
        model=MY_CONFIG.LLM_MODEL,
    )

✅ Using LLM model : nebius/openai/gpt-oss-120b


## Step-7: Query

In [8]:
%%time 

import query_utils

question = "How were Granite models trained?"
query_engine = index.as_query_engine()
query = query_utils.tweak_query(question, MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

Granite models were built using a comprehensive pipeline that began with large‑scale data collection, followed by careful filtering and preprocessing of the code‑related corpora. The resulting datasets were fed into decoder‑only transformer architectures ranging from 3 billion to 34 billion parameters. Training proceeded in multiple stages: an initial pre‑training phase on the curated code data, then a subsequent instruction‑tuning stage that exposed the models to task‑specific prompts for generation, bug fixing, explanation, and documentation. Throughout, the training process incorporated the architectural and hyper‑parameter choices detailed in the model design section, and the entire workflow was documented across the data preparation, model architecture, training, and instruction‑tuning sections of the paper.
CPU times: user 110 ms, sys: 14.2 ms, total: 124 ms
Wall time: 7.85 s


In [9]:
%%time 

import query_utils

question = "What is attention mechanism?"
query_engine = index.as_query_engine()
query = query_utils.tweak_query(question, MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

The attention mechanism takes a query vector together with a collection of key‑value pairs (each also represented as vectors) and produces an output vector. It does this by computing similarity scores between the query and each key, turning those scores into weights, and then forming a weighted sum of the corresponding values. The resulting weighted sum is the attention output.
CPU times: user 35.5 ms, sys: 7.33 ms, total: 42.8 ms
Wall time: 3.77 s


In [10]:
%%time 

import query_utils

question = "When was the moon landing?"
query_engine = index.as_query_engine()
query = query_utils.tweak_query(question, MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

The provided context does not contain information about the date of the moon landing.
CPU times: user 33.2 ms, sys: 4.4 ms, total: 37.6 ms
Wall time: 2.02 s
